In [1]:
import json, os, pickle, random
import numpy as np
from tqdm.notebook import tqdm
import pandas as pd

import torch
from torch.utils.data import Dataset, DataLoader, TensorDataset
from torch.optim.lr_scheduler import ReduceLROnPlateau
import torch.nn as nn

import qiskit
from qiskit import QuantumCircuit, execute
from qiskit.compiler import transpile
from qiskit_aer import AerSimulator, QasmSimulator
from qiskit.converters import circuit_to_dag, dag_to_circuit
from qiskit.quantum_info import SparsePauliOp, Operator
from qiskit.circuit.library import CXGate, RXGate, IGate, ZGate
from qiskit.providers.fake_provider import FakeMontreal, FakeLima

from utils import get_backend_properties_v1

# from blackwater.data.utils import (
#     generate_random_pauli_sum_op,
#     create_estimator_meas_data,
#     circuit_to_graph_data_json,
#     get_backend_properties_v1,
#     encode_pauli_sum_op,
#     create_meas_data_from_estimators
# )

from mlp import MLP1, MLP2, MLP3, encode_data

#from mbd_utils import cal_z_exp, generate_disorder, construct_mbl_circuit, calc_imbalance, modify_and_add_noise_to_model

import matplotlib.pyplot as plt
import seaborn as sns
from noise_utils import AddNoise, RemoveReadoutErrors

In [2]:
from mlp import recursive_dict_loop, count_gates_by_rotation_angle

In [3]:
from mbd_utils import calc_imbalance
import torch
from torch import nn
from sklearn import datasets
import sklearn

In [4]:
def fix_random_seed(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  # if you are using multi-GPU.
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True
    print(f'random seed fixed to {seed}')

In [5]:
backend = FakeLima()
properties = get_backend_properties_v1(backend)

## Local
backend_ideal = QasmSimulator() # Noiseless
backend_noisy = AerSimulator.from_backend(FakeLima()) # Noisy

run_config_ideal = {'shots': 10000, 'backend': backend_ideal, 'name': 'ideal'}
run_config_noisy = {'shots': 10000, 'backend': backend_noisy, 'name': 'noisy'}

In [6]:
backend = FakeLima()
properties = get_backend_properties_v1(backend)

# Local, coherent noise
backend_ideal = QasmSimulator() # Noiseless
backend_noisy_coherent, noise_model = AddNoise(backend=backend).add_coherent_noise(seed=0, theta=np.pi * 0.04, uniform=False, add_depolarization=True)

run_config_ideal = {'shots': 10000, 'backend': backend_ideal, 'name': 'ideal'}
run_config_noisy_coherent = {'shots': 10000, 'backend': backend_noisy_coherent, 'name': 'noisy_coherent'}

random seed fixed to 0
thetas [0.06896594 0.08987335 0.07574548 0.06847204 0.05323803 0.08116545
 0.05498883 0.1120635 ]


In [7]:
backend = FakeLima()
properties = get_backend_properties_v1(backend)

# Local, coherent noise
backend_ideal = QasmSimulator() # Noiseless
backend_noisy_no_readout = RemoveReadoutErrors().remove_readout_errors()[0]

run_config_ideal = {'shots': 10000, 'backend': backend_ideal, 'name': 'ideal'}

# no measurement errors 
run_config_noisy_no_readout = {'shots': 10000, 'backend': backend_noisy_no_readout, 'name': 'noisy_no_readout'}

num_qubit = 4

In [8]:
from qiskit.quantum_info import DensityMatrix, partial_trace
from qiskit import IBMQ, pulse
from qiskit.circuit.library import QFT

In [9]:
def load_circuits(data_dir, f_ext='.json'):
    circuits = []
    ideal_exp_vals = []
    noisy_exp_vals = []
    data_files = [os.path.join(data_dir, f) for f in os.listdir(data_dir) if f.endswith(f_ext)]
    for data_file in tqdm(data_files[1:], leave=True):
        #print(data_file)
        if f_ext == '.json':
            for entry in json.load(open(data_file, 'r')):
                circuits.append(QuantumCircuit.from_qasm_str(entry['circuit']))
                ideal_exp_vals.append(entry['ideal_exp_value'])
                noisy_exp_vals.append(entry['noisy_exp_values'])
        elif f_ext == '.pk':
            for entry in pickle.load(open(data_file, 'rb')):
                circuits.append(entry['circuit'])
                ideal_exp_vals.append(entry['ideal_exp_value'])
                noisy_exp_vals.append(entry['noisy_exp_values'])
    return circuits, ideal_exp_vals, noisy_exp_vals

In [10]:
train_circuits, train_ideal_exp_vals, train_noisy_exp_vals_input = load_circuits('./data/ising_init_from_qasm_no_readout/train/', '.pk')
print(len(train_circuits))

test_circuits, test_ideal_exp_vals, test_noisy_exp_vals_input = load_circuits('./data/ising_init_from_qasm_no_readout/val/', '.pk')
print(len(test_circuits))

  0%|          | 0/14 [00:00<?, ?it/s]

4200


  0%|          | 0/14 [00:00<?, ?it/s]

1400


topology circuit from dag method
<!-- total n of gates
n of cnots
backend properties -->
try training on mukltiple backends
use model noise instead of backend (there is some link between model noise and the backends)
maybe there is something that can be learned from how the noise is modelled some property
use zne to get "worse" noise and let the model learn from it (so maybe it learns some features)



In [11]:
def get_longest_instruction_stream_qc(circuits):
    max_length = 0
    for qc in circuits:
        if max_length < len(qc.data):
            max_length = len(qc.data)
    return max_length

print(get_longest_instruction_stream_qc(train_circuits))

485


In [12]:
gate_dict = {'nothing' : 0}

In [13]:
# python refuses to cooperate :D
def get_dict_max(dict):
    max = -1
    for item in dict:
        if dict[item] > max:
            max = dict[item]
    return max

In [14]:
def encode_data_circuit(circuits, ideal_exp_vals, noisy_exp_vals):
    X = []
    y = []
    i = 0
    for quantum_circuit in circuits:
        features = []
        # converts qc.data into list of operations
        for instr, qargs, _ in quantum_circuit.data:
            gate_name = instr.name
            
            if not (gate_name in gate_dict):
                gate_dict[gate_name] = get_dict_max(gate_dict) + 1
                gate_encoding = gate_dict[gate_name]
            else:
                gate_encoding = gate_dict[gate_name]

            if gate_name == 'barrier':
                break

            gate_params = instr.params if instr.params else []
            qubit_indices = [qarg.index for qarg in qargs]

            feature = []

            feature.append(gate_encoding)
            if type(gate_params) == list:
                for param in gate_params:
                    feature.append(float(param))
            else:
                feature.append(float(gate_params))
            feature.extend(qubit_indices)

            if len(feature) == 2:
                feature = [feature[0], 0, feature[1]]

            for item in feature: features.append (item)

        while (len(features)) < 39:
            features.append(0)

        #features.append(noisy_exp_vals[i][0][0])

        i += 1

        X.append(features)

        #print(X)

    for val in ideal_exp_vals:
        y.append(val[0])

    X = torch.FloatTensor(X)
    y = torch.FloatTensor(y)

    return X, y


In [45]:
X_train, y_train = encode_data_circuit(train_circuits, train_ideal_exp_vals, train_noisy_exp_vals_input)
X_test, y_test = encode_data_circuit(test_circuits, test_ideal_exp_vals, test_noisy_exp_vals_input)

In [46]:
from torch.utils.data import DataLoader
from torch.utils.data.dataset import TensorDataset
train_data = TensorDataset(X_train, y_train)
test_data = TensorDataset(X_test, y_test)

batch_size = 32
train_loader = DataLoader(train_data,
                          batch_size=batch_size,
                          shuffle=True)
test_loader = DataLoader(test_data,
                         batch_size=len(y_test),
                         shuffle=False)

In [47]:
#setup device agnostic code
if torch.cuda.is_available():
  device = "cuda"
else:
  device = "cpu"
device

'cpu'

In [48]:
def train_step(model: torch.nn.Module,
               dataloader,
               loss_fn: torch.nn.Module,
               optimiser: torch.optim.Optimizer,
               device=device):
  model.train()

  train_loss, train_acc = 0, 0

  for batch, (X, y) in enumerate(dataloader):
    X, y = X.to(device), y.to(device)

    y_pred = model(X)

    loss = loss_fn(y_pred, y)
    train_loss += loss.item()

    optimiser.zero_grad()

    loss.backward()

    optimiser.step()

    y_pred_class = torch.argmax(torch.softmax(y_pred, dim=1), dim=1)
    train_acc += ((y_pred_class==y).sum().item()/len(y_pred))

  train_loss = train_loss / len(dataloader)
  train_acc = train_acc / len(dataloader)
  return train_loss, train_acc

In [49]:
def test_step(model: torch.nn.Module,
              dataloader,
              loss_fn: torch.nn.Module,
              device=device):
  model.eval()

  test_loss, test_acc = 0, 0

  with torch.inference_mode():
    for batch, (X, y) in enumerate(dataloader):
      X, y = X.to(device), y.to(device)

      test_pred_logits = model(X)

      loss = loss_fn(test_pred_logits, y)
      test_loss += loss.item()

      test_pred_labels = test_pred_logits.argmax(dim=1)
      test_acc += ((test_pred_labels == y).sum().item()/len(test_pred_labels))

  test_loss = test_loss / len(dataloader)
  test_acc = test_acc / len(dataloader)

  return test_loss, test_acc

In [50]:
from tqdm.auto import tqdm

#create a train function that takes in various model parameters
def train(model: torch.nn.Module,
          train_data,
          test_data,
          optimiser: torch.optim.Optimizer,
          loss_fn: torch.nn.Module,
          epochs: int = 5,
          device=device):

  results = {"train_loss": [],
             "train_acc": [],
             "test_loss": [],
             "test_acc": []}
  for epoch in tqdm(range(epochs)):
    train_loss, train_acc = train_step(model=model,
                                       dataloader=train_data,
                                       loss_fn=loss_fn,
                                       optimiser=optimiser,
                                       device=device)
    test_loss, test_acc = test_step(model=model,
                                    dataloader=test_data,
                                    loss_fn=loss_fn,
                                    device=device)
    if epoch % 10 == 0:
      print(f"Epoch: {epoch}, Train loss: {train_loss:.4f}, Train acc: {train_acc:.4f}, Test loss: {test_loss:.4f}, Test acc: {test_acc:.4f}")

    results["train_loss"].append(train_loss)
    results["train_acc"].append(train_acc)
    results["test_loss"].append(test_loss)
    results["test_acc"].append(test_acc)

In [51]:
class Model1(nn.Module):
  def __init__(self,
               input_shape: int,
               hidden_units: int,
               hidden_layers: int,
               output_shape: int):
    super().__init__()
    self.layers = nn.ModuleDict()
    self.nLayers = hidden_layers

    self.layers['input'] = nn.Linear(input_shape,
                                     hidden_units)

    for i in range(hidden_layers):
      self.layers[f'hidden{i}'] = nn.Linear(hidden_units,
                                            hidden_units)

    self.layers['output'] = nn.Linear(hidden_units,
                                      output_shape)

  def forward(self, x):
    ReLU = nn.ReLU()

    x = ReLU(self.layers['input'](x))

    for i in range(self.nLayers):
      x = ReLU(self.layers[f'hidden{i}'](x))

    x = self.layers['output'](x)

    return x

In [52]:
# redo the input output shapes later
model_main = Model1(input_shape=len(X_train[0]),
                    hidden_units=32,
                    hidden_layers=4,
                    output_shape=1).to(device)

model_main

Model1(
  (layers): ModuleDict(
    (input): Linear(in_features=39, out_features=32, bias=True)
    (hidden0): Linear(in_features=32, out_features=32, bias=True)
    (hidden1): Linear(in_features=32, out_features=32, bias=True)
    (hidden2): Linear(in_features=32, out_features=32, bias=True)
    (hidden3): Linear(in_features=32, out_features=32, bias=True)
    (output): Linear(in_features=32, out_features=1, bias=True)
  )
)

In [53]:
loss_fn = torch.nn.MSELoss()

optimiser = torch.optim.Adam(params=model_main.parameters(),
                            lr=0.1)

In [54]:
EPOCH_COUNT = 100

train(model=model_main,
      train_data=train_loader,
      test_data=test_loader,
      optimiser=optimiser,
      loss_fn=loss_fn,
      epochs=EPOCH_COUNT,
      device=device)

  0%|          | 0/100 [00:00<?, ?it/s]

c:\Users\andre\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\nn\modules\loss.py:535: UserWarning: Using a target size (torch.Size([32])) that is different to the input size (torch.Size([32, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
c:\Users\andre\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\nn\modules\loss.py:535: UserWarning: Using a target size (torch.Size([8])) that is different to the input size (torch.Size([8, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
c:\Users\andre\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\nn\modules\loss.py:535: UserWarning: Using a target size (torch.Size([1400])) that is different to the input size (torch.Size([1400, 1])). This will likely lead to inc

Epoch: 0, Train loss: 11.2009, Train acc: 0.0002, Test loss: 0.0557, Test acc: 0.0007
Epoch: 10, Train loss: 0.0577, Train acc: 0.0002, Test loss: 0.0556, Test acc: 0.0007
Epoch: 20, Train loss: 0.0564, Train acc: 0.0002, Test loss: 0.0552, Test acc: 0.0007
Epoch: 30, Train loss: 0.0565, Train acc: 0.0002, Test loss: 0.0551, Test acc: 0.0007
Epoch: 40, Train loss: 0.0579, Train acc: 0.0002, Test loss: 0.0561, Test acc: 0.0007
Epoch: 50, Train loss: 0.0573, Train acc: 0.0002, Test loss: 0.0559, Test acc: 0.0007
Epoch: 60, Train loss: 0.0573, Train acc: 0.0002, Test loss: 0.0563, Test acc: 0.0007
Epoch: 70, Train loss: 0.0587, Train acc: 0.0002, Test loss: 0.0558, Test acc: 0.0007
Epoch: 80, Train loss: 0.0572, Train acc: 0.0002, Test loss: 0.0552, Test acc: 0.0007
Epoch: 90, Train loss: 0.0587, Train acc: 0.0002, Test loss: 0.0557, Test acc: 0.0007


In [55]:
model_main.eval()
with torch.inference_mode():
  y_preds = model_main(X_test)

from sklearn.metrics import root_mean_squared_error

rms = root_mean_squared_error(y_test, y_preds)
print(rms)

0.23491223


In [56]:
# from utils import circuit_to_graph_data_json
# graph_data = circuit_to_graph_data_json(
#     circuit=pracc_circuits[0],
#     properties=properties,
#     use_qubit_features=True,
#     use_gate_features=True,
# )

In [57]:
# graph_data

In [58]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
rfr = RandomForestRegressor(n_estimators=300)
rfr.fit(X_train, y_train)

RandomForestRegressor(n_estimators=300)

In [59]:
y_pred_rf = rfr.predict(X_test)
rms = root_mean_squared_error(y_test, y_pred_rf)

In [60]:
print(rms)

0.2304767302456263
